In [ ]:
import gemmi as gm
import numpy as np
import matplotlib.pyplot as plt

path = "/Users/alejandro/Desktop/mddocs/processing-analysis/PDBs-Codes/fibrilGen/"
filepath = path + "collagen-molecule.pdb"

st = gm.read_structure(filepath)
cell = st.cell
m0 = st[0]                      # base model

# Shift list
shifts = [[1,0,0], [0,1,0], [-1,0,0], [0,-1,0], [1,1,0], [-1,-1,0], [0,0,1], [0,0,-1], [1,-1,0], [-1,1,0]]
models = []


for shift in shifts:
    m_new = 0

    # Get the translation vector - x||a, constructs over this
    shift_frac = gm.Fractional(shift[0], shift[1], shift[2])

    # convert to Cartesian (Å) using the actual cell (handles non-90° cells too)
    shift_cart = cell.orthogonalize(shift_frac)
    ta, tb, tc = shift_cart.tolist()
    tvec = gm.Vec3(ta, tb, tc)

    # build a Transform: identity rotation + translation vector
    T = gm.Transform(gm.Mat33(), tvec)

    # clone and move
    m_new = m0.clone()
    m_new.transform_pos_and_adp(T)       # moves atoms; ADPs handled correctly (rotation-aware)
    models.append(m_new)

# generate model - joining the 2 models
out = gm.Structure()
out.cell = st.cell
out.spacegroup_hm = st.spacegroup_hm

# Create model 0
out.add_model(gm.Model("0"))
mdl = out[0]

# Function to make a single model
def add_model_into_single(m, model):
    used = {ch.name for ch in model}
    pool = iter("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789")
    for ch in m:
        ch2 = ch.clone()
        if len(ch2.name) != 1 or ch2.name in used:
            # assign a fresh single-char ID
            nid = next(pool)
            while nid in used:
                nid = next(pool)
            ch2.name = nid
        used.add(ch2.name)
        mdl.add_chain(ch2)

add_model_into_single(m0, mdl)
for model in models:
    add_model_into_single(model, mdl)

out.write_pdb(path + "colstruct.pdb")